In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [4]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [5]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [6]:
indep=dataset[[ 'Age', 'EstimatedSalary','Gender_Male' ]]
dep=dataset[ 'Purchased']

In [7]:
indep.shape

(400, 3)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [9]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
x_train=sc.fit_transform(X_train)
x_test=sc.transform(X_test)

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
param_grid = {'criterion':['gini', 'entropy', 'log_loss'],'max_features':['sqrt','log2'],'splitter':['best','random']}
grid = GridSearchCV( DecisionTreeClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
20 fits failed out of a total of 60.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'

  warnings.warn(some_

GridSearchCV(estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_features': ['sqrt', 'log2'],
                         'splitter': ['best', 'random']},
             scoring='f1_weighted', verbose=3)

In [25]:
re=grid.cv_results_
grid_predictions = grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)

C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\base.py:435: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [26]:
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions )

C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test, grid_predictions,average='weighted')
print("The f1 macro value for parameter {}:".format(grid.best_params_),f1_macro)

The f1 macro value for parameter {'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'best'}: 0.1958241578990294


In [28]:
print(cm)

[[ 0 85]
 [ 0 49]]


In [29]:
print(clf_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        85
           1       0.37      1.00      0.54        49

    accuracy                           0.37       134
   macro avg       0.18      0.50      0.27       134
weighted avg       0.13      0.37      0.20       134



In [30]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

C:\Users\ELCOT\Anaconda3\lib\site-packages\sklearn\base.py:435: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


0.5

In [31]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_splitter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.003126,0.006251,0.000000,0.000000,gini,sqrt,best,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.822092,0.906252,0.814409,0.925272,0.924528,0.878511,0.049731,1
1,0.002301,0.004602,0.000601,0.001202,gini,sqrt,random,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.812243,0.849057,0.814409,0.814409,0.885265,0.835077,0.028601,5
2,0.000000,0.000000,0.003127,0.006253,gini,log2,best,"{'criterion': 'gini', 'max_features': 'log2', ...",0.867478,0.849057,0.814409,0.925272,0.850543,0.861352,0.036318,3
3,0.000000,0.000000,0.000000,0.000000,gini,log2,random,"{'criterion': 'gini', 'max_features': 'log2', ...",0.770712,0.849057,0.778067,0.813179,0.801281,0.802459,0.027894,8
4,0.000000,0.000000,0.000000,0.000000,entropy,sqrt,best,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.776290,0.828959,0.813179,0.813179,0.923510,0.831023,0.049377,6
5,0.000601,0.001201,0.000399,0.000798,entropy,sqrt,random,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.864871,0.828959,0.740593,0.887907,0.826499,0.829766,0.050152,7
6,0.001631,0.001998,0.000824,0.001009,entropy,log2,best,"{'criterion': 'entropy', 'max_features': 'log2...",0.829615,0.870047,0.814409,0.869709,0.943041,0.865364,0.044607,2
7,0.000000,0.000000,0.003127,0.006253,entropy,log2,random,"{'criterion': 'entropy', 'max_features': 'log2...",0.888889,0.752940,0.832483,0.886792,0.832483,0.838717,0.049525,4
8,0.000801,0.000981,0.000000,0.000000,log_loss,sqrt,best,"{'criterion': 'log_loss', 'max_features': 'sqr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,9
9,0.000000,0.000000,0.000000,0.000000,log_loss,sqrt,random,"{'criterion': 'log_loss', 'max_features': 'sqr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,10
